In [2]:
#spark.stop()
%run ../connection.python.py

In [3]:
digitalizzazione_201810_df= spark.read.format('parquet').load('hdfs:///user/silvia/digitalizzazione_parziale_navigazione_201810')
digitalizzazione_201810_df.createOrReplaceTempView("digitalizzazione_201810")

In [28]:
digitalizzazione_201810_df.show(10,False)


gb_liv_finale_2018 = spark.sql("select livello_digitalizzazione, count(*) as count from digitalizzazione_201810 group by livello_digitalizzazione")

gb_liv_finale_2018.show(15,False)

+------------------------------------------------+------------------------+------------------------+
|skyid                                           |score_totale_navigazione|livello_digitalizzazione|
+------------------------------------------------+------------------------+------------------------+
|0005eb13fdd2dd6347d3e560c21230692adeceb96c9db2cd|0.0                     |0                       |
|0005f3f18bda8a53a22af40921ad131c0e4e913cbdb5f565|6.0                     |6                       |
|0006530d860a31ca2a4b3539ad9baab4e2d668b4ee0da1a4|6.5                     |7                       |
|0006b83f9d38ac301f2026071b2b816841bab3c09da501cc|3.0                     |3                       |
|000aec8698c204ecadd4011222549f24efc5bc3ac949c457|7.0                     |7                       |
|000effd99733aebfe511e729e39b71cdee0c9edd644891f5|6.0                     |6                       |
|000fd3e0c929462b34a48485da2eb775e19b90b2a467e904|9.0                     |9               

In [5]:
digitalizzazione_201810_df.count()

4506444

In [6]:
digitalizzazione_201709_path= 'hdfs:///user/alessandro/clusterfinalscore_conApp.parquet'

In [7]:
digitalizzazione_201709_df= spark.read.format('parquet').load(digitalizzazione_201709_path)
digitalizzazione_201709_df.printSchema()
digitalizzazione_201709_df.createOrReplaceTempView("digitalizzazione_201709")

root
 |-- skyid: string (nullable = true)
 |-- livello_digitalizzazione: double (nullable = true)
 |-- predictionCMDM: integer (nullable = true)
 |-- scoreCMDM: double (nullable = true)
 |-- cluster: integer (nullable = true)
 |-- cluster_score: integer (nullable = true)
 |-- devices_prediction: integer (nullable = true)
 |-- visite_prediction: integer (nullable = true)
 |-- devices_prediction_score: integer (nullable = true)
 |-- visite_prediction_score: integer (nullable = true)



In [8]:
digitalizzazione_201709_df=digitalizzazione_201709_df.distinct()

In [9]:
import pyspark.sql.functions as F

In [10]:
digitalizzazione_201709_df=digitalizzazione_201709_df.withColumn('digitalizzazione_bin', F.expr("IF(livello_digitalizzazione = 0, 0, IF(livello_digitalizzazione <=3, 1, IF(livello_digitalizzazione <=6, 2, IF(livello_digitalizzazione <=9, 3, IF(livello_digitalizzazione <=12, 4, IF(livello_digitalizzazione <=15, 5, IF(livello_digitalizzazione <=18, 6, IF(livello_digitalizzazione <=21, 7, IF(livello_digitalizzazione <=24, 8, IF(livello_digitalizzazione <=27, 9, IF(livello_digitalizzazione <=30, 10, null)))))))))))"))

In [11]:
digitalizzazione_201709_df.createOrReplaceTempView("digitalizzazione_201709")
digitalizzazione_201709_df.count()

4178841

In [12]:
digitalizzazione_201709_df.show(10,False)

+------------------------------------------------+------------------------+--------------+---------+-------+-------------+------------------+-----------------+------------------------+-----------------------+--------------------+
|skyid                                           |livello_digitalizzazione|predictionCMDM|scoreCMDM|cluster|cluster_score|devices_prediction|visite_prediction|devices_prediction_score|visite_prediction_score|digitalizzazione_bin|
+------------------------------------------------+------------------------+--------------+---------+-------+-------------+------------------+-----------------+------------------------+-----------------------+--------------------+
|015297d80b4b967f8c4ad4fc67d68f4af2293373d1c7c11f|20.184                  |4             |7.184    |4      |6            |4                 |1                |5                       |2                      |7                   |
|042f6438eb89c8d28cd7fb82abef7ced689be9928b77beea|7.184                   |4    

In [25]:
digitalizzazione_201709_df_mod = spark.sql("select skyid, digitalizzazione_bin as livello_vecchio, (cluster_score + (devices_prediction_score + visite_prediction_score)) as livello_digitalizzazione from digitalizzazione_201709")

In [26]:
digitalizzazione_201709_df_mod = digitalizzazione_201709_df_mod.withColumn('livello_finale', F.expr("IF(livello_digitalizzazione = 0, 0, IF(livello_digitalizzazione <=2, 1, IF(livello_digitalizzazione <=4, 2, IF(livello_digitalizzazione <=6, 3, IF(livello_digitalizzazione <=8, 4, IF(livello_digitalizzazione <=10, 5, IF(livello_digitalizzazione <=12, 6, IF(livello_digitalizzazione <=14, 7, IF(livello_digitalizzazione <=16, 8, IF(livello_digitalizzazione <=18, 9, IF(livello_digitalizzazione <=20, 10, null)))))))))))"))
digitalizzazione_201709_df_mod.show(15,False)


+------------------------------------------------+---------------+------------------------+--------------+
|skyid                                           |livello_vecchio|livello_digitalizzazione|livello_finale|
+------------------------------------------------+---------------+------------------------+--------------+
|015297d80b4b967f8c4ad4fc67d68f4af2293373d1c7c11f|7              |13                      |7             |
|042f6438eb89c8d28cd7fb82abef7ced689be9928b77beea|3              |0                       |0             |
|057e00511f17ab51eb83b6c7beb21279bb0d4b48fa6de0c9|8              |14                      |7             |
|06afc0a3a80402fdb1977b57ec11e95356a699621e5bfbfc|10             |20                      |10            |
|06ffe54517237ef007c31f63dc1db834d40c577e59bbf471|3              |5                       |3             |
|08e77482c6de4073a9ea313e41330e6979fb9d227ef52c17|7              |12                      |6             |
|0a79a4ef7edcf616adb485675d60abea747a

In [27]:
digitalizzazione_201709_df_mod.createOrReplaceTempView("digitalizzazione_201709_df_mod")
gb_liv_finale_2017 = spark.sql("select livello_finale, count(*) as count from digitalizzazione_201709_df_mod group by livello_finale")

gb_liv_finale_2017.show(15,False)


gb_liv_vecchio_2017 = spark.sql("select livello_vecchio, count(*) as count from digitalizzazione_201709_df_mod group by livello_vecchio")

gb_liv_vecchio_2017.show(15,False)

+--------------+-------+
|livello_finale|count  |
+--------------+-------+
|6             |295701 |
|3             |276395 |
|5             |293690 |
|9             |329016 |
|4             |257684 |
|8             |309624 |
|7             |330023 |
|10            |480615 |
|2             |100848 |
|0             |1505245|
+--------------+-------+

+---------------+------+
|livello_vecchio|count |
+---------------+------+
|1              |726186|
|6              |331278|
|3              |388745|
|5              |316538|
|9              |354094|
|4              |220934|
|8              |380383|
|7              |405872|
|10             |507456|
|2              |232957|
|0              |314398|
+---------------+------+



In [29]:
summary_confronto=spark.sql("select a.livello_finale as livello_digitalizzazione_2017, b.livello_digitalizzazione as livello_digitalizzazione_2018, count(*) num_skyid from digitalizzazione_201709 a full outer join digitalizzazione_201810 b on a.skyid=b.skyid group by 1,2").persist()

In [30]:
summary_confronto.show(10000,False)

+-----------------------------+-----------------------------+---------+
|livello_digitalizzazione_2017|livello_digitalizzazione_2018|num_skyid|
+-----------------------------+-----------------------------+---------+
|10                           |2                            |423      |
|1                            |0                            |482315   |
|9                            |10                           |70573    |
|4                            |10                           |6424     |
|6                            |1                            |24738    |
|3                            |1                            |45655    |
|7                            |4                            |47354    |
|2                            |2                            |25649    |
|8                            |9                            |34447    |
|9                            |4                            |19330    |
|8                            |3                            |225

In [33]:
new_skyid=spark.sql("select a.livello_finale as livello_digitalizzazione_2017, b.livello_digitalizzazione as livello_digitalizzazione_2018, count(*) num_skyid from digitalizzazione_201709 a left join digitalizzazione_201810 b on a.skyid=b.skyid where a.skyid is NULL group by 1,2").persist()

new_skyid.show(20,False)



AnalysisException: "cannot resolve '`a.livello_finale`' given input columns: [scoreCMDM, digitalizzazione_bin, score_totale_navigazione, devices_prediction, cluster, skyid, visite_prediction_score, devices_prediction_score, livello_digitalizzazione, skyid, livello_digitalizzazione, cluster_score, predictionCMDM, visite_prediction]; line 1 pos 7;\n'Aggregate [unresolvedordinal(1), unresolvedordinal(2)], ['a.livello_finale AS livello_digitalizzazione_2017#327, livello_digitalizzazione#2 AS livello_digitalizzazione_2018#328, count(1) AS num_skyid#329L]\n+- Filter isnull(skyid#23)\n   +- Join LeftOuter, (skyid#23 = skyid#0)\n      :- SubqueryAlias a, `digitalizzazione_201709`\n      :  +- Project [skyid#23, livello_digitalizzazione#24, predictionCMDM#25, scoreCMDM#26, cluster#27, cluster_score#28, devices_prediction#29, visite_prediction#30, devices_prediction_score#31, visite_prediction_score#32, if ((livello_digitalizzazione#24 = cast(0 as double))) 0 else if ((livello_digitalizzazione#2